# Exploring Data
This notebook is aimed to explore cargo repo data

In [2]:
import pandas
import matplotlib
import seaborn
import collections
import itertools

import sys
sys.path.append('..')

from helper import load_data , load_repo

%matplotlib inline

# Load Cargo

In [3]:
cargo = load_repo('Cargo')

In [4]:
cargo_git = cargo[cargo.Repository_URL.notnull()]
cargo_git = cargo_git[cargo_git.Repository_URL.str.contains('github')]
cargo_git = cargo_git.drop_duplicates(subset='Repository_URL')

# Load Dependencies

In [5]:
packages, dependencies = load_data('Cargo')

In [6]:
packages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82335 entries, 0 to 82336
Data columns (total 3 columns):
package    82335 non-null object
version    82335 non-null object
date       82335 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 2.5+ MB


All unique cargo packages and adding a column as package-version combination

In [7]:
unique_cargo_packages = packages.drop_duplicates(subset='package',keep = 'last')
unique_cargo_packages['pack_ver'] = unique_cargo_packages['package']+unique_cargo_packages['version']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


All dependencies

In [8]:
dependencies['pack_ver'] = dependencies['package']+dependencies['version']
all_unique_dependencies = dependencies[dependencies['pack_ver'].isin(unique_cargo_packages['pack_ver'])]
all_unique_dependencies = all_unique_dependencies.drop('pack_ver',axis=1).reset_index().drop('index',axis=1)

In [9]:
all_unique_dependencies.head(10)

,package,version,target,constraint
0,acacia,0.1.2,itertools,^0.5
1,acacia,0.1.2,num,^0.1
2,acacia,0.1.2,rand,^0.3
3,aio,0.0.1,event,*
4,aio,0.0.1,mio,*
5,aio,0.0.1,nix,*
6,advapi32-sys,0.2.0,winapi,^0.2.5
7,alfred,4.0.1,serde_json,^1.0
8,algebloat,0.0.8,algebloat_macros,= 0.0.8
9,allegro_acodec,0.0.8,allegro_acodec-sys,= 0.0.8


# Related Git URLs
finding git url that each package contributor communicates with 

In [10]:
repos = cargo_git[['Name','Repository_URL']].set_index('Name')
repos.head()

,Repository_URL
Name,
acacia,https://github.com/aepsil0n/acacia
abort_on_panic,https://github.com/emk/abort_on_panic-rs
aio,https://github.com/reem/rust-aio
advapi32-sys,https://github.com/retep998/winapi-rs
alfred,https://github.com/kballard/alfred-rs


In [11]:
ser = all_unique_dependencies['target']
all_unique_dependencies['target_url'] = repos.loc[all_unique_dependencies['target']].reset_index()[['Repository_URL']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


For each package which github repositories data should be retrieved

In [12]:
all_unique_dependencies.head(30)

,package,version,target,constraint,target_url
0,acacia,0.1.2,itertools,^0.5,https://github.com/bluss/rust-itertools
1,acacia,0.1.2,num,^0.1,https://github.com/rust-num/num
2,acacia,0.1.2,rand,^0.3,https://github.com/rust-lang-nursery/rand
3,aio,0.0.1,event,*,https://github.com/reem/rust-event
4,aio,0.0.1,mio,*,https://github.com/carllerche/mio
5,aio,0.0.1,nix,*,https://github.com/nix-rust/nix
6,advapi32-sys,0.2.0,winapi,^0.2.5,NaN
7,alfred,4.0.1,serde_json,^1.0,https://github.com/serde-rs/json
8,algebloat,0.0.8,algebloat_macros,= 0.0.8,NaN
9,allegro_acodec,0.0.8,allegro_acodec-sys,= 0.0.8,NaN


# unique repositories
to find all unique repository addresses

In [13]:
final_repo_list = all_unique_dependencies.dropna(subset=['target_url'], how='all').target_url.unique()

# Method 1 
with subprocess the downside is that we have to clone the repository before running get log

In [39]:
import subprocess

# creates new row of dataframe
def newrow(row,project):
    rowdata ={
        "project_name": project,
        "commit_hash": row[0],
        "commit_date": row[1],
        "author_name": row[2],
        "author_email": row[3],
        "committer_name": row[4],
        "committer_email": row[5] }
    return rowdata
    
# get log of a spesific repo
def get_repo_commits(repo):
    
    #clone repository to a bare repo to work with gitlog
    repo_name = repo.split('/')[3] + repo.split('/')[4]
    commands = ["git", "clone" ,"--bare", repo , '.\\tempclone\\'+repo_name]
    process = subprocess.Popen(commands,stdout=subprocess.PIPE)
    output = process.communicate()[0]

    #get git log result by subprocess
    commands = ["git", "log" , '--pretty=format:%H";"%ai";"%an";"%ae";"%cn";"%ce:%#$GLZDH']
    process = subprocess.Popen(commands,  cwd=r'.\\tempclone\\'+repo_name ,stdout=subprocess.PIPE)
    output = process.communicate()[0]
    output = str(output)
    
    return output

# read commit data from gitlog result and add it to dataframe
def add_to_dataset(commits,project,df):
    #find the main text from git log commit string
    commits = commits[2:len(commits)-10]
    for commit in commits.split(':%#$GLZDH\\'):
        #if commit length is 0
        if len(commit) == 0:
            return df

        #split to find the parts and if there is problem with some parts we return 
        rowdata = commit.split('";"')
        if len(rowdata) < 6:
            print(rowdata)
            return df

        #append data to dataframe
        df = df.append(newrow(rowdata,project),ignore_index=True)
        
    return df

# a dataframe to store commit data                                
all_commits = pandas.DataFrame(columns=["project_name","commit_hash","commit_date","author_name","author_email","committer_name","committer_email"])

prevlen = 0

# iterating all repositories to retrieve commit history
for repo in final_repo_list:
    try:
        all_commits = add_to_dataset(get_repo_commits(repo),repo,all_commits)
    except:
        print("except")
        
    print('{} - {} , {}'.format(repo , (len(all_commits) - prevlen), len(all_commits)))
    prevlen = len(all_commits)
    
    if(len(all_commits) >4000):
        break;
    # pauses process for 2 second although its not necessary
    matplotlib.pyplot.pause(2)
    
    
all_commits.to_csv('../data/commitlogs.csv')

https://github.com/bluss/rust-itertools - 1026 , 1026
https://github.com/rust-num/num - 736 , 1762
https://github.com/rust-lang-nursery/rand - 1860 , 3622
https://github.com/reem/rust-event - 59 , 3681
https://github.com/carllerche/mio - 664 , 4345


# Method 2 
with pygithub the downside is that it takes to much time to (although data for each repository there is just one request for all commit history from Github.com)

In [14]:
from github import Github

g = Github("f01369c3dd13beb4d69f9ee115f9835e30002d67")

all_commits = pandas.DataFrame(columns=["project_name","commit_hash","commit_date","author_name","author_email","committer_name","committer_email"])

def new_row(commit,repo):
    row = {
        "project_name": repo,
        "commit_hash": commit.sha,
        "commit_date": commit.commit.committer.date , # if commit.commit.committer is not None else '',
        "author_name": commit.author.name, # if commit.author is not None else '',
        "author_email": commit.author.email, # if commit.author is not None else '',
        "committer_name": commit.committer.name, # if commit.committer is not None else '',
        "committer_email": commit.committer.email # if commit.committer is not None else ''
    }
    return row

prevlen = 0
# iterating all repositories to retrieve commit history
for repo in final_repo_list:
    # try to create repo if exist
    try:
        repos = g.get_repo(repo.split("https://github.com/")[1])
    except:
        print('{} -- has problem'.format(repo.split("https://github.com/")[1]))
        continue
        
    print(repos)
    # try to get commits from repo and add to dataframe
    repo_commits = repos.get_commits()
    print(repo_commits.totalCount)
    for commit in repo_commits:
        print(commit)
        all_commits = all_commits.append(new_row(commit,repo),ignore_index=True)
        
    # print sizes
    print('{} - {} , {}'.format(repo , (len(all_commits) - prevlen), len(all_commits)))
    prevlen = len(all_commits)

Repository(full_name="bluss/rust-itertools")
1026
Commit(sha="8bb5c66a7ed6aa42158a0186f322a472872da6b3")
Commit(sha="841a4b56ae11783c166bc9415d72b479991201c5")
Commit(sha="bfe61670944ab2a32f7b86daae12515de1ec617c")
Commit(sha="1ee4d1622ad7623d9c26502202b576ca121e47a5")
Commit(sha="710d9f248b50a70adbfbff5824b5710d6d315d7a")
Commit(sha="bcb12f090dd0d030cbcd783220c37ceeb4548e2b")
Commit(sha="eb0ab69565d9bbe01e5b7700b0d978bf1c60c47f")
Commit(sha="cb3b0748767d3bb4b8abae636c524ff38fb2eb26")
Commit(sha="12f05afeebb7c32366c75a481fb3ae693b1da95b")
Commit(sha="19eb024a5ccb8e811c9a0a4da9b047895d1d7151")
Commit(sha="16adaf1cf363b34dde4397b94fe9b0955418c84d")
Commit(sha="3e31f7125c9e0eef2d47e1b60b55e55368e9dae8")
Commit(sha="e3d49f420fe86bd2e486f37db499357ac4de34c5")
Commit(sha="8be531516fa53551f48747657e0be1c1aeb964bb")
Commit(sha="04b051a112463aa59b09ce28681e46860f55773e")
Commit(sha="92cf26f185612dc57adf1d0515be7f86040a7952")
Commit(sha="2cb079a29425dc3cd1ce782a4ab9960df457b391")
Commit(sha="b93

KeyboardInterrupt: 

In [40]:
all_commits

,project_name,commit_hash,commit_date,author_name,author_email,committer_name,committer_email
0,https://github.com/bluss/rust-itertools,8bb5c66a7ed6aa42158a0186f322a472872da6b3,2018-08-27 22:11:09 +0200,bluss,bluss@users.noreply.github.com,GitHub,noreply@github.com
1,https://github.com/bluss/rust-itertools,n841a4b56ae11783c166bc9415d72b479991201c5,2018-08-27 09:31:53 +0300,Aleksey Kladov,aleksey.kladov@gmail.com,Aleksey Kladov,aleksey.kladov@gmail.com
2,https://github.com/bluss/rust-itertools,nbfe61670944ab2a32f7b86daae12515de1ec617c,2018-06-30 11:09:15 +0200,bluss,bluss@users.noreply.github.com,GitHub,noreply@github.com
3,https://github.com/bluss/rust-itertools,n1ee4d1622ad7623d9c26502202b576ca121e47a5,2018-06-27 17:36:01 -0400,Jim Turner,git@turner.link,Jim Turner,git@turner.link
4,https://github.com/bluss/rust-itertools,n710d9f248b50a70adbfbff5824b5710d6d315d7a,2018-06-23 13:15:04 +0200,bluss,bluss@users.noreply.github.com,GitHub,noreply@github.com
5,https://github.com/bluss/rust-itertools,nbcb12f090dd0d030cbcd783220c37ceeb4548e2b,2018-06-22 22:52:16 +0200,Mazdak Farrokhzad,twingoow@gmail.com,GitHub,noreply@github.com
6,https://github.com/bluss/rust-itertools,neb0ab69565d9bbe01e5b7700b0d978bf1c60c47f,2018-06-17 15:26:09 +0200,Mazdak Farrokhzad,twingoow@gmail.com,GitHub,noreply@github.com
7,https://github.com/bluss/rust-itertools,ncb3b0748767d3bb4b8abae636c524ff38fb2eb26,2018-06-13 20:09:14 +0200,bluss,bluss@users.noreply.github.com,GitHub,noreply@github.com
8,https://github.com/bluss/rust-itertools,n12f05afeebb7c32366c75a481fb3ae693b1da95b,2018-05-16 18:07:36 -0400,Jake Goulding,jake.goulding@gmail.com,GitHub,noreply@github.com
9,https://github.com/bluss/rust-itertools,n19eb024a5ccb8e811c9a0a4da9b047895d1d7151,2018-05-10 22:12:12 +0200,bluss,bluss@users.noreply.github.com,GitHub,noreply@github.com
